# Header

In [ ]:
# %pip install leafmap
# %pip install geopandas
# %pip install GDAL==3.10.2  # versions varies on your environment    

In [1]:
import os
import leafmap
import geopandas as gpd
from osgeo import gdal

### Setup Map Variables

 - GIS Baselayer - Add a basemap from [KyGisServer](https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services).  In this instance, I will use the Commonweatlh Topography Basemap.
 ```
 https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_TCM_Topo_Base_WGS84WM/MapServer
 ```

 - [Cedar Run-Kentucky River/Stoney Creek-Kentucky River/Lower North Benson Creek-Benson Creek](https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_8_10_12_Digit_Hydrologic_Units_WGS84WM/MapServer/0) 12 digit hydrologic units from KyGisServer

 - Download [3DHP Streamlines](https://opengisdata.ky.gov/datasets/a278069c6db44b63a4fee8670b7a5adf_0/api) from Kentucky's GIS Open Data Portal API Explorer

In [2]:
# add a kentucky basemap
input_url = 'https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_TCM_Topo_Base_WGS84WM/MapServer'
# ky_base_map = input('Please paste WMS: \n') or input_url
ky_base_map = input_url
ky_base_map += '/tile/{z}/{y}/{x}'              # concatenate text for tile layer format
print(f'{ky_base_map}\n')

frankfort_hucs = 'https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_8_10_12_Digit_Hydrologic_Units_WGS84WM/MapServer/0/query?where=Name+IN+%28%27Cedar+Run-Kentucky+River%27%2C+%27Stony+Creek-Kentucky+River%27%2C+%27Lower+North+Benson+Creek-Benson+Creek%27%29&units=esriSRUnit_Foot&outFields=Name%2CHUC12&returnExtentOnly=false&featureEncoding=esriDefault&f=geojson'
gdf = gpd.read_file(frankfort_hucs)

# Save to Geopackage to use externally to clip streams
gdf.to_file('frankfort_hux.gpkg', layer='hucs', driver='GPKG')

https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_TCM_Topo_Base_WGS84WM/MapServer/tile/{z}/{y}/{x}



In [ ]:
# # initiate Map m
# m = leafmap.Map(
#     center=(38.2,-84.9),
#     zoom=11,
#     draw_control=False,
#     measure_control=False,
#     fullscreen_control=False,
#     )

# # add Kentucky's Topography Basemap as a WMTS
# m.add_tile_layer(
#     url=ky_base_map,
#     name='Ky TCM Topo',
#     attribution='<a href="https://kygeonet.ky.gov" target="_blank">DGI</a>'
# )

# # add Lower Kentucky River geometry to map
# m.add_gdf(
#     gdf,
#     layer_type='fill',
#     layer_name='Benson/Cedar/Stony Creeks',
# )

### Get Elevation Data

 - Get DEM Cogs from AWS
 - Clip Tiles to Huc8
 - Create a list of COGs/Save to file
 - Build a Virtual Raster/Export VRT
 - Visualize

Geopackage urls can be copied from [KyFromAbove AWL Explorer](https://kyfromabove.s3.us-west-2.amazonaws.com/index.html#elevation/DEM/TileGrids/). Just right click and copy the link.  This project will use the Phase2 due to availability.
```
https://kyfromabove.s3-us-west-2.amazonaws.com/elevation/DEM/TileGrids/kyfromabove_phase2_5k_dem_grid.gpkg
```


In [ ]:
# Get Elevation Data
input_gpkg = 'https://kyfromabove.s3-us-west-2.amazonaws.com/elevation/DEM/TileGrids/kyfromabove_phase2_5k_dem_grid.gpkg'
# dem_url = input('Paste your Digital Elevation Model Url: \n') or input_gpkg
dem_url = input_gpkg
dem_gdf = gpd.read_file(dem_url)

Create a function to clip tiles to an area of interest.

In [ ]:
def clip_tiles_to_aoi(gdf_tiles, gdf_aoi):
    # convert to wgs84 for plotting
    gdf_tiles = gdf_tiles.to_crs('EPSG:4326')
    gdf_aoi = gdf_aoi.to_crs('EPSG:4326') 
    # clip the tiles to the AOI
    gdf_tiles_clipped = gpd.clip(gdf_tiles, gdf_aoi)
    return gdf_tiles_clipped

# clip tiles
# for simplicity, setup variables
gdf_aoi = gdf  # Watersheds around Frankfort
gdf_tiles = dem_gdf

# clip tiles
dem_gdf_clipped = clip_tiles_to_aoi(gdf_tiles, gdf_aoi)

# Save to fil
dem_gdf_clipped.to_file('dem_gdf_clipped.gpkg', layer='dem_gdf_clipped', driver='GPKG')


Write list of AWS URLs to file

We can use a list or a file

In [ ]:
# Create an Array of COG files
dem_list = []
for index, row in dem_gdf_clipped.iterrows():
    dem_list.append(row['aws_url'])

# for simplicity, setup variables
gdf_aoi = gdf  # Watersheds around Frankfort
gdf_tiles = dem_gdf

# clip tiles
dem_gdf_clipped = clip_tiles_to_aoi(gdf_tiles, gdf_aoi)


#### Create a Digital Elevation Model

In [ ]:
# Set up VRT Options
vrt_options = gdal.BuildVRTOptions(
    resolution='average',
    resampleAlg='bilinear'
)

# set up output VRT file
out_vrt = os.path.join(os.getcwd(), 'dem.vrt')

if os.path.exists(out_vrt):
  print('VRT exists')
else:
  # Build VRT
  vrt = gdal.BuildVRT(out_vrt, dem_list, options=vrt_options)

# Create a COG from VRT
dem_cog = os.path.join(os.getcwd(), 'dem_cog.tif')
if os.path.exists('dem_cog.tif'):
  print('COG exists')
else:
  gdal.Translate(
      'dem_cog.tif',
      out_vrt,
      format='COG',
      creationOptions=[
          'OVERVIEWS=IGNORE_EXISTING',
          'NUM_THREADS=-1',
          'COMPRESS=LZW',
          'BIGTIFF=YES'  # allows for files larger than 4gb
      ]
)

In [ ]:
# Create Map
# initiate Map m
m = leafmap.Map(
    center=(38.2,-84.9),
    zoom=11,
    height="1000px",
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    )

# add Kentucky's Topography Basemap as a WMTS
m.add_tile_layer(
    url=ky_base_map,
    name='Ky TCM Topo',
    attribution='<a href="https://kygeonet.ky.gov" target="_blank">DGI</a>'
)

# add Lower Kentucky River geometry to map
m.add_gdf(
    gdf,
    layer_type='fill',
    layer_name='Benson/Cedar/Stony Creeks',
)

# add cog to map
m.add_raster(
    dem_cog,
    layer_name='Ky 2FT DEM',
    colormap='terrain',
    opacity=0.8
)

m.to_html('frankfort-map.html', add_layer_control=True)
m

In [ ]:
# # initiate Map m
# m2 = leafmap.Map(
#     center=(38.2,-84.9),
#     zoom=11,
#     draw_control=False,
#     measure_control=False,
#     fullscreen_control=False,
#     )

# # add Kentucky's Topography Basemap as a WMTS
# m2.add_tile_layer(
#     url=ky_base_map,
#     name='Ky TCM Topo',
#     attribution='<a href="https://kygeonet.ky.gov" target="_blank">DGI</a>'
# )

# # # add cog to map
# # m2.add_raster(
# #     dem_cog,
# #     layer_name='Ky 2FT DEM',
# #     colormap='terrain',
# #     opacity=0.8
# # )

# # add fill_dep
# m2.add_raster(
#     'fill_dep.tif',
#     layer_name='Fill Depressions',
#     colormap='terrain',
#     opacity=0.8
# )

# # # add fill_dep
# # m2.add_raster(
# #     'fd8_flow_acc.tif',
# #     layer_name='Flow Accumulation',
# #     colormap='terrain',
# #     opacity=0.8
# # )

# # add fill_dep
# m2.add_raster(
#     'extracted_streams.tif',
#     layer_name='Streams',
#     opacity=0.8
# )

# m2.to_html()
# m2
